In [1]:
import os
import sys

# os.path.join(os.path.dirname(__file__)
sys.path.append("../")  # location of config file
sys.path.append("../../") # working directory inside exploration
sys.path.append("../../../") 



import config

os.chdir("../../")

import glob
import numpy as np
from smartprint import smartprint as sprint
from preprocessing.ProcessRaw import ProcessRaw
from smartprint import smartprint as sprint
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
# ! rm -rf dist_plots
# ! mkdir dist_plots

In [4]:
#########################################################################
#################  tryout: smoothing and thresholds  ####################
#########################################################################
# ! pip install peakdetect==1.2

import matplotlib.patches as patches
from scipy.stats import entropy

def temporal_filter_simple(file_list, query_point, n, L):
    """
    sample 2L points
    n: the serial number identifier for query point
    """
    if L == "ALL":
        filename_range = range(len(file_list))
    else:
        filename_range = range(n-L, n+L)
        
    distance_list = []
    
    for i in filename_range:
        if i == n:
            # no need for 0 distance
            continue
        distance_list.append(np.max(np.abs(query_point - np.load(file_list[i]))))
     
    return distance_list



def temporal_filter_advanced(file_list, query_point, n, L):
    """
    sample 2L points
    n: the serial number identifier for query point
    """
    if L == "ALL":
        filename_range = range(len(file_list))
    else:
        filename_range = []
        for i in range(7):  # 7 days
            filename_range.extend(  list (range(n + 96*i - L, n + 96*i + L))  )
            
    distance_list = []
    
    for i in filename_range:
        if i == n:
            # no need for 0 distance
            continue
        distance_list.append(np.max(np.abs(query_point - np.load(file_list[i]))))
     
    return distance_list
    

    
which_func = temporal_filter_simple

for cityname in ["london", "Madrid", "Melbourne"]:
    for scale in [45, 105, 205]:
        for i_o_length in [1]:
            for pred_horiz in [1]: # [1]:
                plt.clf()
                
                ProcessRaw(cityname=cityname, i_o_length=i_o_length, prediction_horizon=pred_horiz, grid_size=scale)
                prefix = ProcessRaw.file_prefix(cityname, i_o_length, pred_horiz, scale)

                file_list = glob.glob("../train_data_all_cities/" + prefix + "/*_x.npy")
                
                # Here we need sorted file so that we can directly find the neighbours
                file_list = sorted(file_list, key=lambda x: int(x.split('-')[-1].split('_x.npy')[0]))

                        
                symmetric_KL_div_list_stacked = []
                for repeat in tqdm(range(30), desc="Repeating N times"):
                    symmetric_KL_div_list = []
                    
                    L_range = np.arange(1, 15*14, 1)
                    BINS = np.arange(0, 7000, 100)
                    
                    # choose query point somewhere at least one week away from the 
                    # boundaries of the dataset; at least 96 * 7 away
                    query_index = np.inf
                    while (not 96 * 10 < query_index < 16000 - 96 * 7):
                        n = int(np.random.rand() * len(file_list))
                        query_point = np.load(file_list[n])
                        query_index = int(file_list[n].split("/")[-1].split("_x.npy")[0].split("-")[-1])
                    
                    distances = which_func(file_list, query_point, n, "ALL")
                    hist_all = np.histogram(distances, bins=BINS)[0]
                    pdf_dist_all = hist_all / hist_all.sum() + 1e-16                        
                    
                    for L in L_range:
    
                        distances = which_func(file_list, query_point, n, L)
                        
                        hist_sample = np.histogram(distances, bins=BINS)[0]
                        pdf_dist_sample = hist_sample / hist_sample.sum() + 1e-16

                        symmetric_KL_div = entropy(pdf_dist_sample, pdf_dist_all) + \
                                           entropy(pdf_dist_all, pdf_dist_sample) 

                        symmetric_KL_div_list.append(symmetric_KL_div)
                    
                        # live plot to see progress as more points are added
                        # plt.plot(symmetric_KL_div_list, alpha=0.5)
                        # plt.savefig("dist_plots/" + "Single point KL-div (whole data, sample) \n"+prefix + ".png")
                        
                    symmetric_KL_div_list_stacked.append(symmetric_KL_div_list)
                    
                    plt.plot(L_range, symmetric_KL_div_list, alpha=0.3)
                    
                plt.plot(L_range, np.median(np.array(symmetric_KL_div_list_stacked), axis=0), \
                     alpha=1, color="black", label="Median")
                plt.xlabel("sample size(N)")
                plt.xticks(list(range(1, 200, 50)), rotation=90, fontsize=8)
                plt.legend()
                plt.title("Single point KL-div whole data vs sample pdf \n"+prefix)
                plt.savefig("dist_plots/Comb-"+str(which_func)\
                            + "Single point KL-div (whole data, sample) \n"+prefix + ".png")
                plt.clf()

london_45_days processed:  91%|█████████▏| 215/235 [00:00<00:00, 14628.52it/s]


start_date, end_date : 2020-02-01 2020-01-31
"Reached end of dates, skipping....Total #dates processed=", len(dates) : Reached end of dates, skipping....Total #dates processed= 215


/home/niskumar/NeurIPS2022-traffic4cast/train_data_all_cities/london-1-1-45-:   0%|          | 0/215 [00:00<?, ?it/s]
london_105_days processed:  91%|█████████▏| 215/235 [00:00<00:00, 13326.27it/s]


start_date, end_date : 2020-02-01 2020-01-31
"Reached end of dates, skipping....Total #dates processed=", len(dates) : Reached end of dates, skipping....Total #dates processed= 215


/home/niskumar/NeurIPS2022-traffic4cast/train_data_all_cities/london-1-1-105-:   0%|          | 0/215 [00:00<?, ?it/s]
london_205_days processed:  91%|█████████▏| 215/235 [00:00<00:00, 14221.34it/s]


...

start_date, end_date : 2020-12-31 2020-12-30
"Reached end of dates, skipping....Total #dates processed=", len(dates) : Reached end of dates, skipping....Total #dates processed= 213


/home/niskumar/NeurIPS2022-traffic4cast/train_data_all_cities/melbourne-1-1-105-:   0%|          | 0/213 [00:00<?, ?it/s]
melbourne_205_days processed:  91%|█████████ | 213/235 [00:00<00:00, 13543.96it/s]


start_date, end_date : 2020-12-31 2020-12-30
"Reached end of dates, skipping....Total #dates processed=", len(dates) : Reached end of dates, skipping....Total #dates processed= 213


/home/niskumar/NeurIPS2022-traffic4cast/train_data_all_cities/melbourne-1-1-205-:   0%|          | 0/213 [00:00<?, ?it/s]
Repeating N times: 100%|██████████| 30/30 [10:22<00:00, 20.75s/it]


<Figure size 640x480 with 0 Axes>

In [5]:
 
which_func = temporal_filter_advanced

for cityname in ["london", "Madrid", "Melbourne"]:
    for scale in [45, 105, 205]:
        for i_o_length in [1]:
            for pred_horiz in [1]: # [1]
                plt.clf()
                
                ProcessRaw(cityname=cityname, i_o_length=i_o_length, prediction_horizon=pred_horiz, grid_size=scale)
                prefix = ProcessRaw.file_prefix(cityname, i_o_length, pred_horiz, scale)

                file_list = glob.glob("../train_data_all_cities/" + prefix + "/*_x.npy")
                
                # Here we need sorted file so that we can directly find the neighbours
                file_list = sorted(file_list, key=lambda x: int(x.split('-')[-1].split('_x.npy')[0]))

                        
                symmetric_KL_div_list_stacked = []
                for repeat in tqdm(range(30), desc="Repeating N times"):
                    symmetric_KL_div_list = []
                    
                    L_range = np.arange(1, 15, 1)
                    BINS = np.arange(0, 7000, 100)
                    
                    # choose query point somewhere at least one week away from the 
                    # boundaries of the dataset; at least 96 * 7 away
                    query_index = np.inf
                    while (not 96 * 10 < query_index < 16000 - 96 * 7):
                        n = int(np.random.rand() * len(file_list))
                        query_point = np.load(file_list[n])
                        query_index = int(file_list[n].split("/")[-1].split("_x.npy")[0].split("-")[-1])
                    
                    distances = which_func(file_list, query_point, n, "ALL")
                    hist_all = np.histogram(distances, bins=BINS)[0]
                    pdf_dist_all = hist_all / hist_all.sum() + 1e-16                        
                    
                    for L in L_range:
    
                        distances = which_func(file_list, query_point, n, L)
                        
                        hist_sample = np.histogram(distances, bins=BINS)[0]
                        pdf_dist_sample = hist_sample / hist_sample.sum() + 1e-16

                        symmetric_KL_div = entropy(pdf_dist_sample, pdf_dist_all) + \
                                           entropy(pdf_dist_all, pdf_dist_sample) 

                        symmetric_KL_div_list.append(symmetric_KL_div)
                    
                        # live plot to see progress as more points are added
                        # plt.plot(symmetric_KL_div_list, alpha=0.5)
                        # plt.savefig("dist_plots/" + "Single point KL-div (whole data, sample) \n"+prefix + ".png")
                        
                    symmetric_KL_div_list_stacked.append(symmetric_KL_div_list)
                    
                    plt.plot(L_range, symmetric_KL_div_list, alpha=0.3)
                    
                plt.plot(L_range, np.median(np.array(symmetric_KL_div_list_stacked), axis=0), \
                     alpha=1, color="black", label="Median")
                plt.xlabel("sample size(N)")
                plt.xticks(list(range(1, 200, 50)), rotation=90, fontsize=8)
                plt.legend()
                plt.title("Single point KL-div whole data vs sample pdf \n"+prefix)
                plt.savefig("dist_plots/Comb-"+str(which_func)\
                            + "Single point KL-div (whole data, sample) \n"+prefix + ".png")
                plt.clf()

london_45_days processed:  91%|█████████▏| 215/235 [00:00<00:00, 14678.05it/s]


start_date, end_date : 2020-02-01 2020-01-31
"Reached end of dates, skipping....Total #dates processed=", len(dates) : Reached end of dates, skipping....Total #dates processed= 215


/home/niskumar/NeurIPS2022-traffic4cast/train_data_all_cities/london-1-1-45-:   0%|          | 0/215 [00:00<?, ?it/s]
london_105_days processed:  91%|█████████▏| 215/235 [00:00<00:00, 14719.74it/s]


start_date, end_date : 2020-02-01 2020-01-31
"Reached end of dates, skipping....Total #dates processed=", len(dates) : Reached end of dates, skipping....Total #dates processed= 215


/home/niskumar/NeurIPS2022-traffic4cast/train_data_all_cities/london-1-1-105-:   0%|          | 0/215 [00:00<?, ?it/s]
london_205_days processed:  91%|█████████▏| 215/235 [00:00<00:00, 17916.19it/s]


...

start_date, end_date : 2020-12-31 2020-12-30
"Reached end of dates, skipping....Total #dates processed=", len(dates) : Reached end of dates, skipping....Total #dates processed= 213


/home/niskumar/NeurIPS2022-traffic4cast/train_data_all_cities/melbourne-1-1-105-:   0%|          | 0/213 [00:00<?, ?it/s]
melbourne_205_days processed:  91%|█████████ | 213/235 [00:00<00:00, 17438.40it/s]


start_date, end_date : 2020-12-31 2020-12-30
"Reached end of dates, skipping....Total #dates processed=", len(dates) : Reached end of dates, skipping....Total #dates processed= 213


/home/niskumar/NeurIPS2022-traffic4cast/train_data_all_cities/melbourne-1-1-205-:   0%|          | 0/213 [00:00<?, ?it/s]
Repeating N times: 100%|██████████| 30/30 [03:03<00:00,  6.10s/it]


<Figure size 640x480 with 0 Axes>

In [4]:
!pwd

/home/niskumar/NeurIPS2022-traffic4cast/exploration
